In [1]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator


batch_size = 128
num_classes = 10
epochs = 12


img_rows, img_cols = 28, 28

(x_train, y_train), (x_test, y_test) = mnist.load_data()


x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)


x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255


y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        fill_mode='nearest')


model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=x_train.shape[0] // batch_size,
                    validation_data=(x_test, y_test),
                    epochs=epochs, verbose=2)

score = model.evaluate(x_test, y_test, verbose=2)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


model.save('model.h5')

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/12
468/468 - 115s - loss: 1.1193 - accuracy: 0.6248 - val_loss: 0.1763 - val_accuracy: 0.9501
Epoch 2/12
468/468 - 106s - loss: 0.5697 - accuracy: 0.8216 - val_loss: 0.1157 - val_accuracy: 0.9661
Epoch 3/12
468/468 - 117s - loss: 0.4420 - accuracy: 0.8646 - val_loss: 0.0927 - val_accuracy: 0.9721
Epoch 4/12
468/468 - 122s - loss: 0.3713 - accuracy: 0.8868 - val_loss: 0.0923 - val_accuracy: 0.9707
Epoch 5/12
468/468 - 95s - loss: 0.3325 - accuracy: 0.8982 - val_loss: 0.0782 - val_accuracy: 0.9746
Epoch 6/12
468/468 - 88s - loss: 0.3108 - accuracy: 0.9068 - val_loss: 0.0714 - val_accuracy: 0.9766
Epoch 7/12
468/468 - 86s - loss: 0.2925 - accuracy: 0.9125 - val_loss: 0.0748 - val_accuracy: 0.9750
Epoch 8/12
468/468 - 98s - loss: 0.2745 - accuracy: 0.9180 - val_loss: 0.0681 - val_accuracy: 0.9765
Epoch 9/12
468/468 - 94s - loss: 0.2634 - accuracy: 0.9210 - val_loss: 0.0666 - val_accuracy: 0.9779
Epoch 10/12

In [ ]:
import io
import socket
import struct
import cv2
from PIL import Image

 
server_socket = socket.socket()
server_socket.bind(('0.0.0.0', 9000))
server_socket.listen(0)
 
# Accept a single connection and make a file-like object out of it
connection = server_socket.accept()[0].makefile('rb')
try:
    while True:
        # Read the length of the image as a 32-bit unsigned int. If the
        # length is zero, quit the loop
        image_len = struct.unpack('<L', connection.read(struct.calcsize('<L')))[0]
        if not image_len:
            break
        # Construct a stream to hold the image data and read the image
        # data from the connection
        image_stream = io.BytesIO()
        image_stream.write(connection.read(image_len))
        # Rewind the stream, open it as an image with PIL and do some
        # processing on it
        image_stream.seek(0)
        image = Image.open(image_stream)
        image.show()
        image.save('mmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmm.jpg')
        print('Image is %dx%d' % image.size)
        image.verify()
        print('Image is verified')
finally:
    connection.close()
    server_socket.close()


Image is 640x480
Image is verified
Image is 640x480
Image is verified


In [4]:
import cv2 as cv
import numpy as np
from tensorflow.keras.models import load_model


img_color = cv.imread('mmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmm.jpg', cv.IMREAD_COLOR)
img_gray = cv.cvtColor(img_color, cv.COLOR_BGR2GRAY)

ret,img_binary = cv.threshold(img_gray, 50, 255, cv.THRESH_BINARY_INV) 
# 50언더는 0값 그 이상은 255를 줘서 라즈베리파이에서 받은 이미지를 이진화 시킨다.

kernel = cv.getStructuringElement( cv.MORPH_RECT, ( 5, 5 ) )
img_binary = cv.morphologyEx(img_binary, cv. MORPH_CLOSE, kernel)

cv.imshow('digit', img_binary)
cv.waitKey(0)

contours, hierarchy = cv.findContours(img_binary, cv.RETR_EXTERNAL, 
                        cv.CHAIN_APPROX_SIMPLE)

for contour in contours:

    x, y, w, h = cv.boundingRect(contour)



    length = max(w, h) + 60
    img_digit = np.zeros((length, length, 1),np.uint8)

    new_x,new_y = x-(length - w)//2, y-(length - h)//2


    img_digit = img_binary[new_y:new_y+length, new_x:new_x+length]

    kernel = np.ones((5, 5), np.uint8)
    img_digit = cv.morphologyEx(img_digit, cv.MORPH_DILATE, kernel)

    cv.imshow('digit', img_digit)
    cv.waitKey(0)

    model = load_model('model.h5')

    img_digit = cv.resize(img_digit, (28, 28), interpolation=cv.INTER_AREA)

    img_digit = img_digit / 255.0

    img_input = img_digit.reshape(1, 28, 28, 1)
    predictions = model.predict(img_input)


    number = np.argmax(predictions)
    print(number)

    cv.rectangle(img_color, (x, y), (x+w, y+h), (255, 255, 0), 2)


    location = (x + int(w *0.5), y - 10)
    font = cv.FONT_HERSHEY_COMPLEX  
    fontScale = 1.2
    cv.putText(img_color, str(number), location, font, fontScale, (0,255,0), 2)
    

    cv.imshow('digit', img_digit)
    cv.waitKey(0)
    

cv.imshow('result', img_color)
cv.waitKey(0)

2
3


-1